<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/LangStudio/SMA_Model_Backtest_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninsta

In [5]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np

# 1. Configuration
SMA_RANGE = range(3, 21)
BB_PERIODS = [10, 20, 30]
SYMBOLS = ["TSLA", "SPY", "QQQ", "NVDA", "META", "MSTR", "COIN", "GLD", "AMD", "SLV", "PLTR", "MSFT", "ORCL", "IWM", "AAPL", "AVGO", "AMZN", "UNH", "NFLX", "MU", "GOOGL", "TSM", "LULU", "CRWV", "GOOG", "IBIT", "JPM", "HOOD", "GDX", "ADBE", "NOW", "APP", "GS", "WOLF", "BABA", "IREN", "COST", "INTC", "LLY", "CRCL", "CVNA", "SNDK", "OKLO", "SMH", "BA", "BMNR", "ASTS", "NBIS", "SOFI", "BE"]

# 2. Optimized Data Cache
print("Fetching fresh data...")
data_cache = {}
for s in SYMBOLS:
    df = yf.download(s, period="5y", interval="1d", progress=False, auto_adjust=True)
    if not df.empty:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        data_cache[s] = df

# 3. Generate Raw Trades with Technical Audit Columns
all_trades_list = []
print("Generating Technical Audit Logs...")

for sma_val in SMA_RANGE:
    for bb_val in BB_PERIODS:
        for symbol, df_orig in data_cache.items():
            df = df_orig.copy()

            # Indicators
            df['SMA_Value'] = ta.sma(df['Close'], length=sma_val)
            bb = ta.bbands(df['Close'], length=bb_val, std=2)
            if bb is None: continue

            bbm_col = [c for c in bb.columns if c.startswith('BBM')][0]
            df['BB_Midpoint'] = bb[bbm_col]

            # Logic: Cross above SMA AND Below BB Midpoint
            df['Prev_Close'] = df['Close'].shift(1)
            df['Prev_SMA'] = df['SMA_Value'].shift(1)

            cond_cross = (df['Close'] > df['SMA_Value']) & (df['Prev_Close'] <= df['Prev_SMA'])
            cond_value = df['Close'] < df['BB_Midpoint']

            # Filter and Label
            trades = df[cond_cross & cond_value].copy()
            if not trades.empty:
                # Forward Returns
                trades['Ret_3D'] = df['Close'].pct_change(3).shift(-3)
                trades['Ret_5D'] = df['Close'].pct_change(5).shift(-5)
                trades['Ret_10D'] = df['Close'].pct_change(10).shift(-10)

                # Metadata
                trades['Symbol'] = symbol
                trades['SMA_Param'] = sma_val
                trades['BB_Param'] = bb_val

                # Selecting Audit Columns
                audit_cols = [
                    'Symbol', 'SMA_Param', 'BB_Param',
                    'Close', 'SMA_Value', 'BB_Midpoint',
                    'Ret_3D', 'Ret_5D', 'Ret_10D'
                ]
                all_trades_list.append(trades[audit_cols])

master_trade_log = pd.concat(all_trades_list)
master_trade_log = master_trade_log.reset_index() # Keep the Date column
print(f"Done! {len(master_trade_log)} total trade instances recorded.")

# VIEW SPECIFIC SYMBOL DETAIL (Example: NVDA)
display(master_trade_log[master_trade_log['Symbol'] == 'NVDA'].head(20))

Fetching fresh data...
Generating Technical Audit Logs...
Done! 87102 total trade instances recorded.


Price,Date,Symbol,SMA_Param,BB_Param,Close,SMA_Value,BB_Midpoint,Ret_3D,Ret_5D,Ret_10D
255,2021-02-24,NVDA,3,10,14.458341,14.292058,14.754982,-0.045331,-0.116853,-0.139764
256,2021-03-01,NVDA,3,10,13.802936,13.583052,14.344885,-0.106309,-0.162444,-0.046667
257,2021-03-09,NVDA,3,10,12.489450,12.158913,13.015729,0.026816,0.061580,0.043969
258,2021-03-26,NVDA,3,10,12.807665,12.641325,12.934751,0.039644,0.089433,0.184571
259,2021-04-23,NVDA,3,10,15.227696,15.121375,15.384484,0.000753,-0.016754,-0.029675
260,2021-05-06,NVDA,3,10,14.487270,14.408715,14.953671,-0.014924,-0.059061,0.006163
261,2021-05-14,NVDA,3,10,14.207960,13.854749,14.286816,-0.012445,0.052570,0.140525
262,2021-07-19,NVDA,3,10,18.737843,18.594080,19.709346,0.043358,0.027383,0.051665
263,2021-08-12,NVDA,3,10,19.860586,19.802382,19.972932,-0.022457,-0.005376,0.108666
264,2021-08-19,NVDA,3,10,19.753820,19.388639,19.819177,0.100768,0.114658,0.131425


In [10]:
# 1. Grouping by Symbol and Strategy Parameters
performance_summary = master_trade_log.groupby(['Symbol', 'SMA_Param', 'BB_Param']).agg(
    Trade_Count=('Ret_3D', 'count'),

    # Win Counts (Raw)
    Wins_3D=('Ret_3D', lambda x: (x > 0).sum()),
    Wins_5D=('Ret_5D', lambda x: (x > 0).sum()),
    Wins_10D=('Ret_10D', lambda x: (x > 0).sum()),

    # Average Returns
    Avg_3D_Ret=('Ret_3D', 'mean'),
    Avg_5D_Ret=('Ret_5D', 'mean'),
    Avg_10D_Ret=('Ret_10D', 'mean')
).reset_index()

# 2. Calculate Win Rates for all three periods
performance_summary['Win_Rate_3D'] = performance_summary['Wins_3D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_5D'] = performance_summary['Wins_5D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_10D'] = performance_summary['Wins_10D'] / performance_summary['Trade_Count']

# 3. Reorder columns for logical flow
display_cols = [
    'Symbol', 'SMA_Param', 'BB_Param', 'Trade_Count',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

# 4. Global Aggregate (Parameter Analysis across all symbols)
global_best = performance_summary.groupby(['SMA_Param', 'BB_Param']).agg({
    'Trade_Count': 'sum',
    'Win_Rate_3D': 'mean',
    'Avg_3D_Ret': 'mean',
    'Win_Rate_5D': 'mean',
    'Avg_5D_Ret': 'mean',
    'Win_Rate_10D': 'mean',
    'Avg_10D_Ret': 'mean'
}).sort_values(by='Win_Rate_10D', ascending=False)

print("--- Top 10 Symbol-Specific Combinations (Sorted by Avg_10D_Ret) ---")
display(performance_summary[display_cols].sort_values('Avg_10D_Ret', ascending=False).head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

print("\n--- Top 10 Parameter Sets Overall (Sorted by 10D Win Rate) ---")
display(global_best.head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- Top 10 Symbol-Specific Combinations (Sorted by 10D Win Rate) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
523,BMNR,4,10,15,40.0%,63.23%,46.7%,200.94%,53.3%,63.55%
520,BMNR,3,10,17,41.2%,54.51%,41.2%,175.00%,47.1%,57.88%
1750,NBIS,20,10,2,50.0%,-3.67%,50.0%,-0.34%,50.0%,37.71%
1745,NBIS,18,10,1,100.0%,5.62%,100.0%,7.41%,100.0%,37.71%
807,CVNA,16,10,6,100.0%,25.98%,83.3%,23.02%,100.0%,27.56%
1266,IREN,14,10,5,80.0%,7.60%,100.0%,19.04%,100.0%,27.36%
2269,SNDK,20,30,2,100.0%,7.70%,100.0%,13.21%,100.0%,24.64%
2268,SNDK,19,30,2,100.0%,7.70%,100.0%,13.21%,100.0%,24.64%
1742,NBIS,17,10,3,66.7%,8.93%,66.7%,12.15%,100.0%,24.15%
1739,NBIS,16,10,2,50.0%,-1.37%,50.0%,-2.22%,100.0%,21.38%



--- Top 10 Parameter Sets Overall (Sorted by 10D Win Rate) ---


In [18]:
# 1. Grouping by Symbol and Strategy Parameters
performance_summary = master_trade_log.groupby(['Symbol', 'SMA_Param', 'BB_Param']).agg(
    Trade_Count=('Ret_3D', 'count'),
    Wins_3D=('Ret_3D', lambda x: (x > 0).sum()),
    Wins_5D=('Ret_5D', lambda x: (x > 0).sum()),
    Wins_10D=('Ret_10D', lambda x: (x > 0).sum()),
    Avg_3D_Ret=('Ret_3D', 'mean'),
    Avg_5D_Ret=('Ret_5D', 'mean'),
    Avg_10D_Ret=('Ret_10D', 'mean')
).reset_index()

# 2. Calculate Win Rates
performance_summary['Win_Rate_3D'] = performance_summary['Wins_3D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_5D'] = performance_summary['Wins_5D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_10D'] = performance_summary['Wins_10D'] / performance_summary['Trade_Count']

# Define display columns for consistency
display_cols = [
    'Symbol', 'SMA_Param', 'BB_Param', 'Trade_Count',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

# --- SECTION A: Best Parameter Set PER Symbol ---
# Sort by Symbol and Win Rate, then take the top 1 for each ticker
best_per_symbol = performance_summary.sort_values(
    ['Symbol', 'Win_Rate_10D', 'Avg_10D_Ret'],
    ascending=[True, False, False]
).groupby('Symbol').head(1)

print("--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---")
display(best_per_symbol[display_cols].sort_values('Win_Rate_10D', ascending=False).head(25).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

# --- SECTION B: Top 10 Parameter Sets OVERALL (Across All Symbols) ---
global_best = performance_summary.groupby(['SMA_Param', 'BB_Param']).agg({
    'Trade_Count': 'sum',
    'Win_Rate_3D': 'mean',
    'Avg_3D_Ret': 'mean',
    'Win_Rate_5D': 'mean',
    'Avg_5D_Ret': 'mean',
    'Win_Rate_10D': 'mean',
    'Avg_10D_Ret': 'mean'
}).sort_values(by='Win_Rate_10D', ascending=False)

print("\n--- [B] TOP 10 PARAMETER SETS OVERALL (Sorted by 10D Win Rate) ---")
display(global_best.head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
308,ASTS,19,20,3,100.0%,10.23%,100.0%,12.94%,100.0%,12.54%
188,AMZN,14,10,8,37.5%,0.95%,87.5%,2.13%,100.0%,4.26%
764,CRWV,18,20,1,100.0%,2.82%,100.0%,12.24%,100.0%,20.67%
807,CVNA,16,10,6,100.0%,25.98%,83.3%,23.02%,100.0%,27.56%
387,BA,11,10,1,100.0%,0.24%,100.0%,9.43%,100.0%,8.31%
560,BMNR,17,10,1,100.0%,6.16%,100.0%,18.79%,100.0%,6.82%
702,CRCL,13,10,1,100.0%,7.75%,100.0%,4.23%,100.0%,21.12%
2370,SPY,19,20,3,66.7%,1.43%,100.0%,3.04%,100.0%,3.67%
2293,SOFI,11,10,2,50.0%,0.95%,50.0%,13.11%,100.0%,13.19%
2268,SNDK,19,30,2,100.0%,7.70%,100.0%,13.21%,100.0%,24.64%



--- [B] TOP 10 PARAMETER SETS OVERALL (Sorted by 10D Win Rate) ---


In [11]:
# --- Symbol Drill-Down Section ---
target_symbol = "NVDA"  # Change this to any symbol from your list

print(f"--- Top 10 Parameter Combinations for {target_symbol} (Sorted by Avg_10D_Ret) ---")

# Filter the performance_summary we created above
symbol_drill_down = performance_summary[performance_summary['Symbol'] == target_symbol]

# Display the results
display(symbol_drill_down[display_cols].sort_values('Avg_10D_Ret', ascending=False).head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- Top 10 Parameter Combinations for NVDA (Sorted by Avg_10D_Ret) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
1881,NVDA,11,30,28,67.9%,1.10%,71.4%,1.87%,64.3%,5.76%
1878,NVDA,10,30,31,64.5%,1.02%,67.7%,1.72%,64.5%,5.56%
1884,NVDA,12,30,28,64.3%,0.83%,60.7%,1.31%,67.9%,5.03%
1876,NVDA,9,30,37,64.9%,1.22%,62.2%,1.70%,59.5%,5.01%
1890,NVDA,14,30,28,71.4%,2.06%,57.1%,1.47%,60.7%,4.90%
1887,NVDA,13,30,25,72.0%,1.90%,64.0%,2.11%,64.0%,4.84%
1880,NVDA,11,20,28,60.7%,0.73%,67.9%,1.51%,64.3%,4.69%
1891,NVDA,15,10,9,66.7%,1.43%,44.4%,1.31%,55.6%,4.65%
1883,NVDA,12,20,28,50.0%,0.01%,57.1%,1.27%,75.0%,4.63%
1873,NVDA,8,30,39,66.7%,1.54%,61.5%,1.43%,61.5%,4.47%


In [14]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Prepare the Aggregate Data (if not already done in Cell 2)
param_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': 'mean',
    'Ret_5D': 'mean',
    'Ret_10D': 'mean'
}).reset_index()

# 2. Define the horizons we want to visualize
horizons = [('3D', 'Ret_3D'), ('5D', 'Ret_5D'), ('10D', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Average Returns) ---")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_table = param_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    # Create the visualization using Pandas Styler for a clean Colab output
    print(f"\n{label} Horizon Average Return:")
    display(pivot_table.style.background_gradient(cmap='RdYlGn', axis=None)
            .format("{:.2%}")
            .set_caption(f"Average {label} Return by SMA and BB Period"))

# Optional: If you prefer a more "visual" graphical heatmap using Seaborn:
# plt.figure(figsize=(10, 6))
# sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap="RdYlGn")
# plt.title("10D Return Surface")
# plt.show()

--- PARAMETER SURFACE HEATMAPS (Average Returns) ---

3D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.47%,0.42%,0.32%
4,0.37%,0.28%,0.20%
5,-0.09%,0.19%,0.12%
6,-0.24%,0.20%,0.22%
7,-0.25%,0.11%,0.18%
8,-0.16%,0.19%,0.29%
9,-0.16%,0.18%,0.29%
10,nan%,0.31%,0.38%
11,0.93%,0.38%,0.46%



5D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.85%,0.36%,0.28%
4,0.88%,0.22%,0.20%
5,-0.15%,0.10%,0.14%
6,-0.39%,0.07%,0.20%
7,-0.28%,0.08%,0.22%
8,-0.33%,0.08%,0.29%
9,-0.16%,0.19%,0.32%
10,nan%,0.32%,0.43%
11,0.77%,0.34%,0.47%



10D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,1.06%,1.03%,1.04%
4,0.93%,0.83%,0.85%
5,0.44%,0.67%,0.77%
6,0.34%,0.87%,0.99%
7,0.64%,1.01%,1.14%
8,0.91%,1.13%,1.28%
9,0.78%,1.13%,1.32%
10,nan%,1.13%,1.37%
11,0.82%,1.26%,1.47%


In [15]:
# 1. Prepare the Win Rate Aggregate Data
# We calculate the mean of the boolean (Return > 0) to get the percentage of wins
win_rate_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': lambda x: (x > 0).mean(),
    'Ret_5D': lambda x: (x > 0).mean(),
    'Ret_10D': lambda x: (x > 0).mean()
}).reset_index()

# 2. Define the horizons for visualization
horizons = [('3-Day', 'Ret_3D'), ('5-Day', 'Ret_5D'), ('10-Day', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---")
print("Focus: Probability of a positive return across the portfolio.")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_win_rate = win_rate_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    print(f"\n{label} Win Rate Surface:")
    # Using a 0.5 (50%) midpoint for the gradient helps highlight 'Edge' vs 'Coin Flip'
    display(pivot_win_rate.style.background_gradient(cmap='RdYlGn', vmin=0.45, vmax=0.65, axis=None)
            .format("{:.1%}")
            .set_caption(f"{label} Win Rate by SMA and BB Period"))

--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---
Focus: Probability of a positive return across the portfolio.

3-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,50.9%,53.0%,52.3%
4,50.1%,52.4%,52.0%
5,48.8%,52.0%,51.6%
6,48.0%,52.1%,52.4%
7,47.8%,52.2%,52.5%
8,48.0%,52.7%,53.6%
9,48.3%,52.5%,53.8%
10,nan%,53.4%,54.8%
11,52.3%,53.7%,55.2%



5-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,50.1%,52.1%,51.8%
4,50.0%,51.5%,51.6%
5,48.4%,50.6%,51.0%
6,48.4%,51.4%,51.8%
7,49.0%,51.8%,52.1%
8,49.2%,52.4%,52.9%
9,49.9%,52.4%,53.0%
10,nan%,53.3%,53.9%
11,51.8%,53.0%,54.0%



10-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,52.8%,53.6%,53.4%
4,51.8%,52.6%,52.8%
5,50.7%,51.9%,52.1%
6,50.5%,52.5%,52.9%
7,51.0%,53.2%,53.6%
8,52.0%,53.8%,54.2%
9,52.5%,54.5%,55.2%
10,nan%,54.6%,55.4%
11,56.3%,55.8%,56.4%


In [13]:
# Rename columns for the CSV to match your previous format
csv_export = param_perf.rename(columns={'SMA_Param': 'SMA', 'BB_Param': 'BB'})

print("--- COPY AND PASTE THIS TO GEMINI ---")
print(csv_export.to_csv(index=False))

--- COPY AND PASTE THIS TO GEMINI ---
SMA,BB,Total_Trades,Wins_3D,Avg_3D,Wins_5D,Avg_5D,Wins_10D,Avg_10D
3,10,4280,2187,0.0047378757233413026,2151,0.008456464739269726,2267,0.010566967508870949
3,20,4390,2333,0.004225830798544758,2294,0.003646937006394012,2358,0.010310069033171056
3,30,4335,2272,0.0031584369463549277,2250,0.002827128410658464,2319,0.010392646030886167
4,10,3306,1660,0.003736809350183828,1658,0.008833726893456335,1718,0.009281430276676338
4,20,3581,1882,0.002780018095338754,1849,0.002191654794785759,1887,0.00825024785388204
4,30,3556,1850,0.0019565174090893428,1836,0.001953341143272934,1879,0.008466790062603988
5,10,2599,1273,-0.0008841996868236558,1263,-0.0014734736475749358,1322,0.004406296184444702
5,20,3057,1592,0.0018955545364872574,1549,0.0010236421281940977,1591,0.0067041157505208365
5,30,3067,1585,0.0012119826642038283,1568,0.0014106136825464892,1602,0.0076565556757764094
6,10,2026,976,-0.0024142534092207557,983,-0.003894644724076122,1026,0.003388320686915572
6,